<a href="https://colab.research.google.com/github/RusAl84/text_analitic/blob/master/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%964_%D0%A2%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Частотный анализ текста с помощью TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

tfidf=tf∗idf

idf=log((N+1)/(Nw+1))+1

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

In [ ]:
texts = "\u041F\u043E\u0447\u0435\u043C\u0443 \u0431\u043E\u043B\u044C\u0448\u0438\u043D\u0441\u0442\u0432\u043E DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0441\u043E\u0437\u0434\u0430\u044E\u0442\u0441\u044F \u043D\u0430 Ethereum \u041E\u043F\u0443\u0431\u043B\u0438\u043A\u043E\u0432\u0430\u043D\u043E: 17/02/2021\u0410\u0432\u0442\u043E\u0440: \u0414\u043C\u0438\u0442\u0440\u0438\u0439 \u0415\u0433\u043E\u0440\u043E\u0432\u041A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u044F: \u041A\u0430\u0440\u0442\u0430 \u0441\u043E\u043A\u0440\u043E\u0432\u0438\u0449 DeFi, \u041D\u043E\u0432\u0438\u0447\u043E\u043A\u041E\u0441\u0442\u0430\u0432\u0438\u0442\u044C \u043A\u043E\u043C\u043C\u0435\u043D\u0442\u0430\u0440\u0438\u0439 \u041F\u043E\u0447\u0435\u043C\u0443 \u0431\u043E\u043B\u044C\u0448\u0438\u043D\u0441\u0442\u0432\u043E DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0441\u043E\u0437\u0434\u0430\u044E\u0442\u0441\u044F \u043D\u0430 Ethereum \u041E\u0431\u043B\u043E\u0436\u043A\u0430 \u0421\u043E\u0434\u0435\u0440\u0436\u0430\u043D\u0438\u0435\t \u0412\u0441\u043F\u043E\u043C\u0438\u043D\u0430\u0435\u043C \u043E\u0441\u043D\u043E\u0432\u043D\u044B\u0435 \u043F\u043E\u043D\u044F\u0442\u0438\u044F \u041A\u0442\u043E \u043E\u0431\u0441\u043B\u0443\u0436\u0438\u0432\u0430\u0435\u0442 DeFi \u043D\u0430 Ethereum Ethereum \u0438 \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u0435 DeFi \u0421\u043A\u043E\u043B\u044C\u043A\u043E DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u043D\u0430 Ethereum \u0417\u0430\u043A\u043B\u044E\u0447\u0435\u043D\u0438\u0435 \u0421\u043E\u0432\u043C\u0435\u0441\u0442\u043D\u043E \u0441 \u043F\u0430\u0440\u0442\u043D\u0435\u0440\u0430\u043C\u0438 \u0438\u0437 Monolithos, \u043C\u044B \u043F\u0440\u043E\u0434\u043E\u043B\u0436\u0430\u0435\u043C \u0446\u0438\u043A\u043B \u043C\u0430\u0442\u0435\u0440\u0438\u0430\u043B\u043E\u0432 \u043F\u0440\u043E \u0434\u0435\u0446\u0435\u043D\u0442\u0440\u0430\u043B\u0438\u0437\u043E\u0432\u0430\u043D\u043D\u044B\u0435 \u0444\u0438\u043D\u0430\u043D\u0441\u044B. \u0421\u0435\u0433\u043E\u0434\u043D\u044F \u043C\u044B \u0440\u0430\u0437\u0431\u0435\u0440\u0435\u043C\u0441\u044F, \u043F\u043E\u0447\u0435\u043C\u0443 \u0431\u043E\u043B\u044C\u0448\u0438\u043D\u0441\u0442\u0432\u043E DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0440\u0430\u0431\u043E\u0442\u0430\u044E\u0442 \u043D\u0430 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435 Ethereum.  \u0414\u0435\u043D\u044C\u0433\u0438 \u043F\u043E\u0434 \u0443\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u0438\u0435\u043C \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C: \u0427\u0442\u043E \u0442\u0430\u043A\u043E\u0435 DeFi \u0417\u0430\u0447\u0435\u043C \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C DeFi, \u0435\u0441\u043B\u0438 \u0435\u0441\u0442\u044C \u0431\u0430\u043D\u043A\u0438 \u0427\u0442\u043E \u043E\u0431\u0449\u0435\u0433\u043E \u0443 DeFi \u0438 \u043A\u0440\u0438\u043F\u0442\u043E\u0432\u0430\u043B\u044E\u0442\u044B \u041F\u043E\u0447\u0435\u043C\u0443 \u0431\u043E\u043B\u044C\u0448\u0438\u043D\u0441\u0442\u0432\u043E DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0441\u043E\u0437\u0434\u0430\u044E\u0442\u0441\u044F \u043D\u0430 Ethereum \u0412\u0441\u043F\u043E\u043C\u0438\u043D\u0430\u0435\u043C \u043E\u0441\u043D\u043E\u0432\u043D\u044B\u0435 \u043F\u043E\u043D\u044F\u0442\u0438\u044F \u0421\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442 \u2014 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0430, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438 \u0441\u043E\u0432\u0435\u0440\u0448\u0430\u0435\u0442 \u0434\u0435\u0439\u0441\u0442\u0432\u0438\u044F \u043F\u0440\u0438 \u0434\u043E\u0441\u0442\u0438\u0436\u0435\u043D\u0438\u0438 \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u043D\u044B\u0445 \u0443\u0441\u043B\u043E\u0432\u0438\u0439. \u0423\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0449\u0438\u0435 \u0442\u043E\u043A\u0435\u043D\u044B \u2014 \u0442\u043E\u043A\u0435\u043D\u044B, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0434\u0430\u044E\u0442 \u043F\u0440\u0430\u0432\u043E \u0433\u043E\u043B\u043E\u0441\u0430 \u0437\u0430 \u0432\u043D\u0435\u0441\u0435\u043D\u0438\u0435 \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u0439 \u0432 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0435. \u0414\u043E\u0445\u043E\u0434\u043D\u043E\u0435 \u0444\u0435\u0440\u043C\u0435\u0440\u0441\u0442\u0432\u043E \u2014 \u0441\u0442\u0440\u0430\u0442\u0435\u0433\u0438\u044F \u0437\u0430\u0440\u0430\u0431\u043E\u0442\u043A\u0430, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u0443\u0447\u0438\u0442\u044B\u0432\u0430\u0435\u0442 \u043D\u0435 \u0442\u043E\u043B\u044C\u043A\u043E \u043F\u0440\u043E\u0446\u0435\u043D\u0442\u044B \u0434\u043E\u0445\u043E\u0434\u043D\u043E\u0441\u0442\u0438, \u043D\u043E \u0438 \u0440\u0430\u0437\u0434\u0430\u0447\u0443 \u0443\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0449\u0438\u0445 \u0442\u043E\u043A\u0435\u043D\u043E\u0432. \u041B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u044C \u2014 \u0441\u0440\u0435\u0434\u0441\u0442\u0432\u0430 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u0439 \u043D\u0430 \u0441\u0447\u0435\u0442\u0430\u0445 \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442\u043E\u0432 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u0430. \u041F\u043E\u0441\u0442\u0430\u0432\u043A\u0430 \u043B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u0438 \u2014 \u0434\u0435\u043F\u043E\u0437\u0438\u0442\u044B \u0432 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u044B. \u041A\u0442\u043E \u043E\u0431\u0441\u043B\u0443\u0436\u0438\u0432\u0430\u0435\u0442 DeFi \u043D\u0430 Ethereum \u0411\u043B\u043E\u043A\u0447\u0435\u0439\u043D Ethereum \u0432\u043A\u043B\u044E\u0447\u0430\u0435\u0442 \u0432 \u0441\u0435\u0431\u044F \u0438\u043D\u0444\u0440\u0430\u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u0443 \u0438\u0437 \u0431\u043E\u043B\u044C\u0448\u043E\u0433\u043E \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u0430 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u043E\u0432 \u0438 \u0440\u0430\u0437\u043B\u0438\u0447\u043D\u044B\u0445 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432, \u0438\u043C\u0435\u044E\u0449\u0438\u0445 \u043E\u0442\u043D\u043E\u0448\u0435\u043D\u0438\u0435 \u043A \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0443.  \u0424\u043E\u043D\u0434\u044B. \u041E\u043D\u0438 \u043F\u043E\u043C\u043E\u0433\u0430\u044E\u0442 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u0430\u043C \u044D\u043A\u0441\u043F\u0435\u0440\u0442\u0438\u0437\u043E\u0439 \u0438 \u0433\u0440\u0430\u043D\u0442\u0430\u043C\u0438. \u041D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, Ethereum Foundation, \u043A\u043E\u0442\u043E\u0440\u044B\u0439 \u043E\u0441\u043D\u043E\u0432\u0430\u043D \u0441\u043E\u0437\u0434\u0430\u0442\u0435\u043B\u0435\u043C Ethereum \u0412\u0438\u0442\u0430\u043B\u0438\u043A\u043E\u043C \u0411\u0443\u0442\u0435\u0440\u0438\u043D\u044B\u043C. \u042D\u0442\u0430 \u043D\u0435\u043A\u043E\u043C\u043C\u0435\u0440\u0447\u0435\u0441\u043A\u0430\u044F \u043E\u0440\u0433\u0430\u043D\u0438\u0437\u0430\u0446\u0438\u044F \u0437\u0430\u043D\u0438\u043C\u0430\u0435\u0442\u0441\u044F \u043F\u043E\u0434\u0434\u0435\u0440\u0436\u043A\u043E\u0439 \u043A\u043E\u043C\u044C\u044E\u043D\u0438\u0442\u0438 \u0438 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u043E\u0432 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432. \u0421 2014 \u0433\u043E\u0434\u0430 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u044F \u043A\u0430\u0436\u0434\u044B\u0439 \u0433\u043E\u0434 \u043E\u0440\u0433\u0430\u043D\u0438\u0437\u043E\u0432\u044B\u0432\u0430\u0435\u0442 Devcon \u2014 \u043A\u043E\u043D\u0444\u0435\u0440\u0435\u043D\u0446\u0438\u044E \u0434\u043B\u044F \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u043E\u0432 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043D\u0430 Ethereum.   \u0424\u043E\u043D\u0434 \u043F\u043E\u0441\u0442\u043E\u044F\u043D\u043D\u043E \u043E\u043A\u0430\u0437\u044B\u0432\u0430\u0435\u0442 \u0444\u0438\u043D\u0430\u043D\u0441\u043E\u0432\u0443\u044E \u043F\u043E\u043C\u043E\u0449\u044C \u0432 \u0432\u0438\u0434\u0435 \u0433\u0440\u0430\u043D\u0442\u043E\u0432. \u0424\u0438\u043D\u0430\u043D\u0441\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u0435 \u043F\u043E\u043B\u0443\u0447\u0430\u043B\u0438 \u043A\u0430\u043A DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u044B, \u0442\u0430\u043A \u0438 \u0434\u0440\u0443\u0433\u0438\u0435 \u0438\u043D\u0444\u0440\u0430\u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u043D\u044B\u0435 \u044D\u043B\u0435\u043C\u0435\u043D\u0442\u044B, \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u044B\u0435 \u0434\u043B\u044F \u0440\u0430\u0431\u043E\u0442\u044B \u0441 DeFi: \u043A\u043E\u0448\u0435\u043B\u044C\u043A\u0438 \u0434\u043B\u044F \u0440\u0430\u0431\u043E\u0442\u044B \u0441 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F\u043C\u0438, \u0438\u0441\u0441\u043B\u0435\u0434\u043E\u0432\u0430\u043D\u0438\u044F \u0438 \u043F\u0440\u0435\u0434\u043B\u043E\u0436\u0435\u043D\u0438\u044F \u043F\u043E \u043C\u0430\u0441\u0448\u0442\u0430\u0431\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044E \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0430, \u043E\u043F\u0442\u0438\u043C\u0438\u0437\u0430\u0446\u0438\u0438 \u043C\u0430\u0439\u043D\u0438\u043D\u0433\u0430 \u0438 \u0443\u043B\u0443\u0447\u0448\u0435\u043D\u0438\u044E \u0438\u043D\u0442\u0435\u0440\u0444\u0435\u0439\u0441\u0430. \u041D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u0432 \u0430\u0432\u0433\u0443\u0441\u0442\u0435 2018 \u0433\u043E\u0434\u0430 Ethereum Foundation \u0432\u044B\u0434\u0435\u043B\u0438\u043B \u0433\u0440\u0430\u043D\u0442\u044B \u043D\u0430 $100 \u0442\u044B\u0441 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u0430\u043C \u0434\u0435\u0446\u0435\u043D\u0442\u0440\u0430\u043B\u0438\u0437\u043E\u0432\u0430\u043D\u043D\u043E\u0439 \u0431\u0438\u0440\u0436\u0438 Uniswap \u0438 $20 \u0442\u044B\u0441 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u0430\u043C \u043A\u043E\u0448\u0435\u043B\u044C\u043A\u0430 Gnosis Safe, \u043F\u0440\u0435\u0434\u043D\u0430\u0437\u043D\u0430\u0447\u0435\u043D\u043D\u043E\u0433\u043E \u0434\u043B\u044F \u0440\u0430\u0431\u043E\u0442\u044B \u0441 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F\u043C\u0438 \u043D\u0430 Ethereum.  \u0410\u0443\u0434\u0438\u0442\u043E\u0440\u0441\u043A\u0438\u0435 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438. \u041E\u043D\u0438 \u043F\u0440\u043E\u0432\u0435\u0440\u044F\u044E\u0442 \u043A\u043E\u0434 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043D\u0430 \u043E\u0448\u0438\u0431\u043A\u0438 \u0438 \u0443\u044F\u0437\u0432\u0438\u043C\u043E\u0441\u0442\u0438. \u042D\u0442\u043E \u043F\u043E\u043C\u043E\u0433\u0430\u0435\u0442 \u043A\u0430\u043A \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u0430\u043C \u043F\u0440\u043E\u0435\u043A\u0442\u0430, \u0442\u0430\u043A \u0438 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u044F\u043C:  \u0420\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u0438 \u043F\u043E\u043D\u0438\u043C\u0430\u044E\u0442, \u043A\u0430\u043A\u0438\u0435 \u043E\u0448\u0438\u0431\u043A\u0438 \u0438\u043C \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E \u0438\u0441\u043F\u0440\u0430\u0432\u0438\u0442\u044C. \u041F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0438 \u043E\u0446\u0435\u043D\u0438\u0432\u0430\u044E\u0442 \u0440\u0438\u0441\u043A\u0438 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u0438\u044F \u043F\u0440\u043E\u0435\u043A\u0442\u0430 \u0438 \u0438\u043D\u0432\u0435\u0441\u0442\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F \u0432 \u043D\u0435\u0433\u043E.  \u041F\u0440\u043E\u0435\u043A\u0442\u044B \u0434\u043B\u044F \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0430 Ethereum \u043D\u0430\u043F\u0438\u0441\u0430\u043D\u044B \u043D\u0430 \u044F\u0437\u044B\u043A\u0435 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F Solidity. \u041A\u0440\u0443\u043F\u043D\u044B\u0435 \u0430\u0443\u0434\u0438\u0442\u043E\u0440\u0441\u043A\u0438\u0435 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438, \u0442\u0430\u043A\u0438\u0435 \u043A\u0430\u043A Trail of Bits, PepperSec \u0438 Mixbytes, \u0443\u0436\u0435 \u0438\u043C\u0435\u044E\u0442 \u043E\u043F\u044B\u0442 \u0440\u0430\u0431\u043E\u0442\u044B \u0441 \u044D\u0442\u0438\u043C \u044F\u0437\u044B\u043A\u043E\u043C, \u043F\u043E\u044D\u0442\u043E\u043C\u0443 \u0443\u0447\u0430\u0441\u0442\u0432\u0443\u044E\u0442 \u0432 \u043F\u0440\u043E\u0432\u0435\u0440\u043A\u0435 \u043A\u043E\u0434\u0430 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043D\u0430 \u044D\u0442\u043E\u043C \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435.  \u041A\u043E\u043C\u043F\u0430\u043D\u0438\u0438, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043F\u0440\u043E\u0432\u043E\u0434\u0438\u043B\u0438 \u0430\u0443\u0434\u0438\u0442\u044B \u0434\u043B\u044F \u0442\u043E\u043F-3 DeFi \u043D\u0430 Ethereum \u041A\u043E\u043C\u043F\u0430\u043D\u0438\u0438, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043F\u0440\u043E\u0432\u043E\u0434\u0438\u043B\u0438 \u0430\u0443\u0434\u0438\u0442\u044B \u0434\u043B\u044F \u0442\u043E\u043F-3 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043F\u043E \u043E\u0431\u044A\u0435\u043C\u0443 \u0437\u0430\u0431\u043B\u043E\u043A\u0438\u0440\u043E\u0432\u0430\u043D\u043D\u044B\u0445 \u0441\u0440\u0435\u0434\u0441\u0442\u0432. \u0414\u0440\u0443\u0433\u043E\u0439 \u043F\u0440\u0438\u043C\u0435\u0440 \u2014 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u044F ConsenSys, \u0441\u043E\u0437\u0434\u0430\u043D\u043D\u0430\u044F \u043E\u0434\u043D\u0438\u043C \u0438\u0437 \u043E\u0441\u043D\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u0439 Ethereum \u0414\u0436\u043E\u0437\u0435\u0444\u043E\u043C \u041B\u044E\u0431\u0438\u043D\u044B\u043C. ConsenSys \u043F\u0440\u043E\u0432\u043E\u0434\u0438\u0442 \u0430\u0443\u0434\u0438\u0442\u044B \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043D\u0430 Ethereum, \u0440\u0430\u0437\u0440\u0430\u0431\u0430\u0442\u044B\u0432\u0430\u0435\u0442 \u0441\u0432\u043E\u0438 \u043F\u0440\u043E\u0435\u043A\u0442\u044B \u0438 \u0444\u0438\u043D\u0430\u043D\u0441\u0438\u0440\u0443\u0435\u0442 \u0434\u0440\u0443\u0433\u0438\u0445 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u043E\u0432 \u0432 \u043F\u0430\u0440\u0442\u043D\u0435\u0440\u0441\u0442\u0432\u0435 \u0441 Ethereum Foundation. \u041F\u043E\u0434\u0440\u0430\u0437\u0434\u0435\u043B\u0435\u043D\u0438\u0435 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438, \u043A\u043E\u0442\u043E\u0440\u043E\u0435 \u0441\u043F\u0435\u0446\u0438\u0430\u043B\u0438\u0437\u0438\u0440\u0443\u0435\u0442\u0441\u044F \u043D\u0430 \u0430\u0443\u0434\u0438\u0442\u0430\u0445, \u043D\u0430\u0437\u044B\u0432\u0430\u0435\u0442\u0441\u044F ConsenSys Diligence. \u041E\u043D\u043E \u043E\u043F\u0443\u0431\u043B\u0438\u043A\u043E\u0432\u0430\u043B\u043E \u043D\u0430 github \u0431\u0430\u0437\u0443 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043F\u0440\u043E\u0448\u043B\u0438 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0430\u0443\u0434\u0438\u0442\u044B \u043A\u0430\u043A \u0443 \u043D\u0438\u0445, \u0442\u0430\u043A \u0438 \u0443 \u0434\u0440\u0443\u0433\u0438\u0445 \u0430\u0443\u0434\u0438\u0442\u043E\u0440\u0441\u043A\u0438\u0445 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0439. \u0412\u0441\u0435\u0433\u043E \u0432 \u0442\u0430\u0431\u043B\u0438\u0446\u0435 154 \u043F\u0440\u043E\u0435\u043A\u0442\u0430 \u0441\u0430\u043C\u043E\u0439 \u0440\u0430\u0437\u043D\u043E\u0439 \u043D\u0430\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u043D\u043E\u0441\u0442\u0438. \u0412 \u043D\u0435\u0439 \u043C\u043E\u0436\u043D\u043E \u043F\u043E\u0441\u043C\u043E\u0442\u0440\u0435\u0442\u044C, \u043A\u0430\u043A\u0438\u0435 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438 \u043F\u0440\u043E\u0432\u043E\u0434\u0438\u043B\u0438 \u0430\u0443\u0434\u0438\u0442\u044B, \u043A\u043E\u0433\u0434\u0430 \u0438 \u0441\u043A\u043E\u043B\u044C\u043A\u043E \u0430\u0443\u0434\u0438\u0442\u043E\u0432 \u0431\u044B\u043B\u043E \u043F\u0440\u043E\u0439\u0434\u0435\u043D\u043E \u0443 \u043A\u043E\u043D\u043A\u0440\u0435\u0442\u043D\u043E\u0433\u043E DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u0430, \u0438\u0437\u0443\u0447\u0438\u0442\u044C \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u044B \u044D\u0442\u0438\u0445 \u0430\u0443\u0434\u0438\u0442\u043E\u0432.  \u0411\u0430\u0437\u0430 \u0434\u0430\u043D\u043D\u044B\u0445 ConsenSys Diligence \u0441\u043E \u0441\u043F\u0438\u0441\u043A\u043E\u043C \u0430\u0443\u0434\u0438\u0442\u043E\u0432 \u0422\u0430\u0431\u043B\u0438\u0446\u0430 ConsenSys Diligence \u0441 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u0430\u043C\u0438 \u0438 \u0441\u0441\u044B\u043B\u043A\u0430\u043C\u0438 \u043D\u0430 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0430\u0443\u0434\u0438\u0442\u044B. \u0421\u043E\u0441\u0442\u0430\u0432\u043B\u0435\u043D\u0430 \u0432 \u0430\u043B\u0444\u0430\u0432\u0438\u0442\u043D\u043E\u043C \u043F\u043E\u0440\u044F\u0434\u043A\u0435. \u0412 \u0433\u0440\u0430\u0444\u0435 \xABMax Bounty Payout\xBB \u0443\u043A\u0430\u0437\u0430\u043D\u043E \u043C\u0430\u043A\u0441\u0438\u043C\u0430\u043B\u044C\u043D\u043E\u0435 \u0432\u043E\u0437\u043D\u0430\u0433\u0440\u0430\u0436\u0434\u0435\u043D\u0438\u0435, \u043A\u043E\u0442\u043E\u0440\u043E\u0435 \u043F\u0440\u0435\u0434\u043B\u0430\u0433\u0430\u044E\u0442 \u0441\u043E\u0437\u0434\u0430\u0442\u0435\u043B\u0438 \u043F\u0440\u043E\u0435\u043A\u0442\u0430 \u0432 \u043E\u0431\u043C\u0435\u043D \u043D\u0430 \u043D\u0430\u0445\u043E\u0436\u0434\u0435\u043D\u0438\u0435 \u043E\u0448\u0438\u0431\u043E\u043A \u0432 \u043A\u043E\u0434\u0435 \u0421\u0442\u0440\u0430\u0445\u043E\u0432\u044B\u0435 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u044B. \u041E\u043D\u0438 \u0434\u0430\u044E\u0442 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u044F\u043C \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E\u0441\u0442\u044C \u0437\u0430\u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u0442\u044C \u0441\u0432\u043E\u0438 \u0430\u043A\u0442\u0438\u0432\u044B \u043E\u0442 \u043E\u0448\u0438\u0431\u043E\u043A \u0432 \u043A\u043E\u0434\u0435 \u0438\u043B\u0438 \u043E\u0442 \u0432\u0437\u043B\u043E\u043C\u0430. \u0412 \u0441\u043B\u0443\u0447\u0430\u0435 \u043D\u0430\u0441\u0442\u0443\u043F\u043B\u0435\u043D\u0438\u044F \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0433\u043E \u0441\u043B\u0443\u0447\u0430\u044F, \u043D\u0430 \u0441\u0447\u0435\u0442\u0443 \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442\u0430 \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0433\u043E \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F \u0434\u043E\u043B\u0436\u043D\u044B \u0431\u044B\u0442\u044C \u0434\u0435\u043D\u044C\u0433\u0438 \u0434\u043B\u044F \u0432\u044B\u043F\u043B\u0430\u0442\u044B \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043A\u0438. \u0427\u0435\u043C \u0431\u043E\u043B\u044C\u0448\u0435 \u0434\u0435\u043D\u0435\u0433 \u0435\u0441\u0442\u044C \u0432 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0438, \u0442\u0435\u043C \u0431\u043E\u043B\u044C\u0448\u0443\u044E \u0441\u0443\u043C\u043C\u0443 \u043C\u043E\u0436\u043D\u043E \u0437\u0430\u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u0442\u044C. \u041D\u0430 \u0441\u0447\u0435\u0442\u0430\u0445 \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u044B\u0445 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043D\u0430 Ethereum \u0435\u0441\u0442\u044C \u0441\u0440\u0435\u0434\u0441\u0442\u0432\u0430 \u0434\u043B\u044F \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u043D\u0438\u044F \u043E\u0442\u043D\u043E\u0441\u0438\u0442\u0435\u043B\u044C\u043D\u043E \u043A\u0440\u0443\u043F\u043D\u044B\u0445, \u0434\u043B\u044F \u043E\u0431\u044B\u0447\u043D\u044B\u0445 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u0439, \u0441\u0443\u043C\u043C. \u041D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u0432 \u0444\u0435\u0432\u0440\u0430\u043B\u0435 2021 \u0433\u043E\u0434\u0430 \u043E\u0431\u0449\u0438\u0439 \u043E\u0431\u044A\u0435\u043C \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u0432 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0438 Nexus Mutual \u0441\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442 420 \u0442\u044B\u0441 ETH \u0438 8,7 \u043C\u043B\u043D \u0441\u0442\u0435\u0439\u0431\u043B\u043A\u043E\u0438\u043D\u043E\u0432 \u0434\u043E\u043B\u043B\u0430\u0440\u0430 \u0421\u0428\u0410 DAI, \u0430 \u0432 Cover Protocol \u2014 24,6 \u043C\u043B\u043D DAI.  \u0418\u043D\u0442\u0435\u0440\u0444\u0435\u0439\u0441 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F Nexus Mutual \u0421\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0435 \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u0435 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0432 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0438 Nexus Mutual. \u041D\u0430 \u0441\u043A\u0440\u0438\u043D\u0448\u043E\u0442\u0435 \u0441\u043F\u0438\u0441\u043E\u043A \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442\u043E\u0432 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043C\u043E\u0436\u043D\u043E \u0437\u0430\u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u0442\u044C, \u0441\u0442\u043E\u0438\u043C\u043E\u0441\u0442\u044C \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u043D\u0438\u044F \u0438 \u0440\u0430\u0437\u043C\u0435\u0440 \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u0432 ETH \u0438 DAI \u041E\u0431\u0449\u0438\u0439 \u043E\u0431\u044A\u0435\u043C \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u2014 \u044D\u0442\u043E \u0441\u0443\u043C\u043C\u0430 \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u0434\u043B\u044F \u0432\u0441\u0435\u0445 \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442\u043E\u0432, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043C\u043E\u0436\u043D\u043E \u0437\u0430\u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u0442\u044C \u0432 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0438. \u0418 \u0443 \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u043D\u0438\u044F \u043A\u043E\u043D\u043A\u0440\u0435\u0442\u043D\u043E\u0433\u043E \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442\u0430 \u0440\u0430\u0437\u043C\u0435\u0440 \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u043C\u043E\u0436\u0435\u0442 \u0441\u043E\u0441\u0442\u0430\u0432\u0438\u0442\u044C \u043A\u0430\u043A \u043D\u0435\u0441\u043A\u043E\u043B\u044C\u043A\u043E \u0442\u044B\u0441\u044F\u0447 DAI, \u0442\u0430\u043A \u0438 \u0441\u043E\u0442\u043D\u0438 \u0442\u044B\u0441\u044F\u0447 DAI.   \u0412 \u0444\u0435\u0432\u0440\u0430\u043B\u0435 2021 \u0433\u043E\u0434\u0430 \u043D\u0430 \u0434\u0440\u0443\u0433\u0438\u0445 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0430\u0445 \u043D\u0435\u0442 \u043F\u0440\u0438\u043C\u0435\u0440\u043E\u0432 DeFi-\u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0439 \u0434\u043B\u044F \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u043D\u0438\u044F \u0441 \u043F\u043E\u0434\u043E\u0431\u043D\u044B\u043C\u0438 \u043F\u043E\u043A\u0430\u0437\u0430\u0442\u0435\u043B\u044F\u043C\u0438 \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F.  \u0418\u043D\u0442\u0435\u0440\u0444\u0435\u0439\u0441 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F Cover Protocol \u0421\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0435 \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u0435 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0432 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0438 Cover Protocol. \u041D\u0430 \u0441\u043A\u0440\u0438\u043D\u0448\u043E\u0442\u0435 \u0441\u043F\u0438\u0441\u043E\u043A \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442\u043E\u0432 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043C\u043E\u0436\u043D\u043E \u0437\u0430\u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u0442\u044C, \u0441\u0442\u043E\u0438\u043C\u043E\u0441\u0442\u044C \u0438\u0445 \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043A\u0438 \u0438 \u0440\u0430\u0437\u043C\u0435\u0440 \u0441\u0442\u0440\u0430\u0445\u043E\u0432\u043E\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u0432 DAI Ethereum \u0438 \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u0435 DeFi \u041F\u0435\u0440\u0432\u044B\u0439 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442 \u043D\u0430\u0447\u0430\u043B \u0440\u0430\u0431\u043E\u0442\u0443 \u043D\u0430 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435 Ethereum \u0432 \u043A\u043E\u043D\u0446\u0435 2017 \u0433\u043E\u0434\u0430. \u041A \u044D\u0442\u043E\u043C\u0443 \u0432\u0440\u0435\u043C\u0435\u043D\u0438 Ethereum \u0443\u0436\u0435 \u043E\u0431\u043B\u0430\u0434\u0430\u043B \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u044B\u043C\u0438 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u043C\u0438 \u0445\u0430\u0440\u0430\u043A\u0442\u0435\u0440\u0438\u0441\u0442\u0438\u043A\u0430\u043C\u0438, \u043F\u0440\u0435\u0438\u043C\u0443\u0449\u0435\u0441\u0442\u0432\u0430\u043C\u0438 \u0434\u043B\u044F \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u044F \u044D\u0442\u043E\u0439 \u0441\u0444\u0435\u0440\u044B:  \u041D\u0430\u043B\u0438\u0447\u0438\u0435 \u0435\u0434\u0438\u043D\u044B\u0445 \u0441\u0442\u0430\u043D\u0434\u0430\u0440\u0442\u043E\u0432 \u0434\u043B\u044F \u0442\u043E\u043A\u0435\u043D\u043E\u0432 \u2014 ERC20, \u043A\u043E\u0442\u043E\u0440\u044B\u0439 \u043F\u0440\u0438\u043C\u0435\u043D\u044F\u0435\u0442\u0441\u044F \u0432 DeFi-\u043F\u0440\u043E\u0442\u043E\u043A\u043E\u043B\u0430\u0445 \u043D\u0430 Ethereum. \u0411\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u0441\u0442\u0430\u043D\u0434\u0430\u0440\u0442\u0443 ERC20, \u0442\u043E\u043A\u0435\u043D \u043E\u0434\u043D\u043E\u0433\u043E \u043F\u0440\u043E\u0435\u043A\u0442\u0430 \u043C\u043E\u0436\u043D\u043E \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C \u0432 \u0434\u0440\u0443\u0433\u0438\u0445 \u043F\u0440\u043E\u0435\u043A\u0442\u0430\u0445; \u042F\u0437\u044B\u043A \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F Solidity, \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0438\u0432\u0430\u044E\u0449\u0438\u0439 \u043F\u0440\u043E\u0441\u0442\u043E\u0442\u0443 \u0441\u043E\u0437\u0434\u0430\u043D\u0438\u044F \u0442\u043E\u043A\u0435\u043D\u043E\u0432 \u0438 \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442\u043E\u0432 \u043D\u0430 Ethereum. \u042D\u0442\u043E\u0442 \u044F\u0437\u044B\u043A \u043E\u0431\u0449\u0438\u0439 \u0434\u043B\u044F \u0432\u0441\u0435\u0445 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043D\u0430 \u044D\u0442\u043E\u043C \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435. \u0412 \u043E\u0442\u043A\u0440\u044B\u0442\u043E\u043C \u0434\u043E\u0441\u0442\u0443\u043F\u0435 \u0435\u0441\u0442\u044C \u0438\u043D\u0441\u0442\u0440\u0443\u043A\u0446\u0438\u0438 \u043E \u0442\u043E\u043C, \u043A\u0430\u043A \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C\u0441\u044F Solidity \u0438 \u043F\u043E\u0441\u0442\u0440\u043E\u0438\u0442\u044C \u0441\u0432\u043E\u0439 \u043F\u0440\u043E\u0435\u043A\u0442 \u043D\u0430 \u043D\u0435\u043C. \u0415\u0441\u043B\u0438 \u043F\u043E\u0441\u043C\u043E\u0442\u0440\u0435\u0442\u044C \u043D\u0430 \u0438\u0441\u0442\u043E\u0440\u0438\u044E \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u044F DeFi, \u0442\u043E \u043C\u044B \u0443\u0432\u0438\u0434\u0438\u043C, \u0447\u0442\u043E \u0432\u0441\u0435 \u043D\u043E\u0432\u044B\u0435 \u0438\u0434\u0435\u0438 \u0438 \u043D\u0430\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u0438\u044F \u0431\u044B\u043B\u0438 \u0432\u043F\u0435\u0440\u0432\u044B\u0435 \u0440\u0435\u0430\u043B\u0438\u0437\u043E\u0432\u0430\u043D\u044B \u043D\u0430 Ethereum. \u041F\u043E \u0434\u0430\u043D\u043D\u044B\u043C \u0441\u0430\u0439\u0442\u0430 defipulse.com, \u0432\u0435\u0441\u044C \u0442\u043E\u043F-10 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0441 \u043D\u0430\u0438\u0431\u043E\u043B\u044C\u0448\u0438\u043C \u043E\u0431\u044A\u0435\u043C\u043E\u043C \u0441\u0440\u0435\u0434\u0441\u0442\u0432 \u043D\u0430 \u0441\u0447\u0435\u0442\u0430\u0445 \u0441\u043E\u0441\u0442\u043E\u0438\u0442 \u0438\u0437 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043D\u0430 Ethereum. \u0411\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u043D\u0430\u043B\u0438\u0447\u0438\u044E \u0441\u0442\u0430\u043D\u0434\u0430\u0440\u0442\u043E\u0432 \u0434\u043B\u044F \u0442\u043E\u043A\u0435\u043D\u043E\u0432 \u0438 \u044F\u0437\u044B\u043A\u0430 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F Solidity, Ethereum \u0441\u0442\u0430\u043B \u0443\u0434\u043E\u0431\u043D\u043E\u0439 \u043F\u043B\u0430\u0442\u0444\u043E\u0440\u043C\u043E\u0439 \u0434\u043B\u044F \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u043E\u0432 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0432 \u0441\u0444\u0435\u0440\u0435 DeFi.  \u0422\u043E\u043F-10 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043F\u043E \u043E\u0431\u044A\u0435\u043C\u0443 \u0441\u0440\u0435\u0434\u0441\u0442\u0432 \u043D\u0430 \u0441\u0430\u0439\u0442\u0435 DeFI Pulse \u0422\u043E\u043F-10 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u043F\u043E \u043E\u0431\u044A\u0435\u043C\u0443 \u0441\u0440\u0435\u0434\u0441\u0442\u0432 \u043D\u0430 \u0441\u0447\u0435\u0442\u0430\u0445 \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442\u043E\u0432. \u0418\u0441\u0442\u043E\u0447\u043D\u0438\u043A: defipulse.com \u041F\u0435\u0440\u0432\u044B\u043C DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u043C \u0441\u0442\u0430\u043B MakerDAO. \u0412 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0438 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0438 \u043F\u043E\u043B\u0443\u0447\u0438\u043B\u0438 \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E\u0441\u0442\u044C \u0432\u044B\u043F\u0443\u0441\u043A\u0430\u0442\u044C \u0441\u0442\u0435\u0439\u0431\u043B\u043A\u043E\u0438\u043D\u044B \u0434\u043E\u043B\u043B\u0430\u0440\u0430 \u0421\u0428\u0410 \u043F\u043E\u0434 \u0437\u0430\u043B\u043E\u0433 \u042D\u0444\u0438\u0440\u0430. \u041F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0435 \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u043F\u043E \u043F\u0440\u0438\u043D\u0446\u0438\u043F\u0443 \u043A\u0440\u0435\u0434\u0438\u0442\u043E\u0432\u0430\u043D\u0438\u044F: \u0447\u0435\u043B\u043E\u0432\u0435\u043A \u0431\u0435\u0440\u0435\u0442 \u0441\u0442\u0435\u0439\u0431\u043B\u043A\u043E\u0438\u043D\u044B \u0432 \u043A\u0440\u0435\u0434\u0438\u0442 \u043F\u043E\u0434 \u0437\u0430\u043B\u043E\u0433 \u043A\u0440\u0438\u043F\u0442\u043E\u0432\u0430\u043B\u044E\u0442\u044B. \u0421\u0442\u0435\u0439\u0431\u043B\u043A\u043E\u0438\u043D\u044B \u043E\u0441\u043D\u043E\u0432\u0430\u043D\u044B \u043D\u0430 \u0441\u0442\u0430\u043D\u0434\u0430\u0440\u0442\u0435 ERC20, \u0430 \u0438\u0445 \u0432\u044B\u043F\u0443\u0441\u043A \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0438\u0432\u0430\u0435\u0442 \u0441\u043C\u0430\u0440\u0442-\u043A\u043E\u043D\u0442\u0440\u0430\u043A\u0442, \u043D\u0430\u043F\u0438\u0441\u0430\u043D\u043D\u044B\u0439 \u043D\u0430 Solidity.   Uniswap \u2014 \u043F\u0435\u0440\u0432\u044B\u0439 \u0443\u0441\u043F\u0435\u0448\u043D\u044B\u0439 \u043F\u0440\u0438\u043C\u0435\u0440 \u0440\u0435\u0430\u043B\u0438\u0437\u0430\u0446\u0438\u0438 \u0434\u0435\u0446\u0435\u043D\u0442\u0440\u0430\u043B\u0438\u0437\u043E\u0432\u0430\u043D\u043D\u043E\u0439 \u0431\u0438\u0440\u0436\u0438 \u0432 DeFi. \u041F\u0440\u043E\u0435\u043A\u0442 \u0441\u043F\u043E\u043D\u0441\u0438\u0440\u043E\u0432\u0430\u043B\u0441\u044F Ethereum Foundation. \u0424\u043E\u043D\u0434 \u0432\u044B\u0434\u0435\u043B\u0438\u043B \u0433\u0440\u0430\u043D\u0442, \u0431\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u043A\u043E\u0442\u043E\u0440\u043E\u043C\u0443 \u0443 Uniswap \u043D\u0430 \u043C\u043E\u043C\u0435\u043D\u0442 \u0437\u0430\u043F\u0443\u0441\u043A\u0430 2 \u043D\u043E\u044F\u0431\u0440\u044F 2018 \u0433\u043E\u0434\u0430 \u0443\u0436\u0435 \u0431\u044B\u043B\u0430 \u043B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u044C \u0438 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0438 \u043C\u043E\u0433\u043B\u0438 \u043F\u0440\u043E\u0442\u0435\u0441\u0442\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u0435\u0435 \u0440\u0430\u0431\u043E\u0442\u0443.  Compound \u2014 \u0437\u0430\u0434\u0430\u043B \u0442\u0440\u0435\u043D\u0434 \u043D\u0430 \xAB\u0414\u043E\u0445\u043E\u0434\u043D\u043E\u0435 \u0444\u0435\u0440\u043C\u0435\u0440\u0441\u0442\u0432\u043E\xBB. \u0420\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u0438 \u0438\u0441\u043A\u0430\u043B\u0438 \u0441\u0440\u0435\u0434\u0441\u0442\u0432\u043E \u0434\u043B\u044F \u0440\u0435\u0448\u0435\u043D\u0438\u044F \u043F\u0440\u043E\u0431\u043B\u0435\u043C\u044B \u043E\u0442\u0441\u0443\u0442\u0441\u0442\u0432\u0438\u044F \u043B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u0438. \u0412 \u043C\u0430\u0435 2020 \u0433\u043E\u0434\u0430 \u043F\u0440\u043E\u0435\u043A\u0442 \u043D\u0430\u0447\u0430\u043B \u0432\u043E\u0437\u043D\u0430\u0433\u0440\u0430\u0436\u0434\u0430\u0442\u044C \u043F\u043E\u0441\u0442\u0430\u0432\u0449\u0438\u043A\u043E\u0432 \u043B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u0438 \u0440\u0430\u0437\u0434\u0430\u0447\u0435\u0439 \u0443\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0449\u0438\u0445 \u0442\u043E\u043A\u0435\u043D\u043E\u0432 COMP, \u0432 \u0434\u043E\u043F\u043E\u043B\u043D\u0435\u043D\u0438\u0435 \u043A \u043F\u0440\u043E\u0446\u0435\u043D\u0442\u0430\u043C \u0434\u043E\u0445\u043E\u0434\u043D\u043E\u0441\u0442\u0438. \u041F\u043E \u0434\u0430\u043D\u043D\u044B\u043C dappradar.com, \u0437\u0430 \u043D\u0435\u0434\u0435\u043B\u044E \u0441 \u043C\u043E\u043C\u0435\u043D\u0442\u0430 \u0437\u0430\u043F\u0443\u0441\u043A\u0430 \u0442\u043E\u043A\u0435\u043D\u0430 COMP \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0443\u043D\u0438\u043A\u0430\u043B\u044C\u043D\u044B\u0445 \u0430\u043A\u0442\u0438\u0432\u043D\u044B\u0445 \u043A\u043E\u0448\u0435\u043B\u044C\u043A\u043E\u0432 \u0432 \u0441\u0435\u0442\u0438 Ethereum \u0443\u0432\u0435\u043B\u0438\u0447\u0438\u043B\u043E\u0441\u044C \u043D\u0430 800%. \u0412\u043F\u0435\u0440\u0432\u044B\u0435 \u0442\u0430\u043A\u0443\u044E \u0441\u0438\u0441\u0442\u0435\u043C\u0443 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u0438\u043B \u043F\u0440\u043E\u0435\u043A\u0442 \u0434\u043B\u044F \u0441\u043E\u0437\u0434\u0430\u043D\u0438\u044F \u0434\u0435\u0440\u0438\u0432\u0430\u0442\u0438\u0432\u043E\u0432 Synthetix, \u0442\u0430\u043A\u0436\u0435 \u0440\u0430\u0431\u043E\u0442\u0430\u044E\u0449\u0438\u0439 \u043D\u0430 Ethereum. \u041D\u043E \u0438\u043C\u0435\u043D\u043D\u043E \u0443\u0441\u043F\u0435\u0445 Compound \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u0441\u043E\u0431\u044B\u0442\u0438\u0435\u043C, \u043F\u043E\u0441\u043B\u0435 \u043A\u043E\u0442\u043E\u0440\u043E\u0433\u043E \u0441\u0438\u0441\u0442\u0435\u043C\u0430 \u0440\u0430\u0441\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u0438\u044F \u0442\u043E\u043A\u0435\u043D\u043E\u0432 \u0441\u0442\u0430\u043B\u0430 \u0438\u0437\u0432\u0435\u0441\u0442\u043D\u043E\u0439 \u0441\u0440\u0435\u0434\u0438 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u0439 \u0438 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u043E\u0432.  Yearn Finance \u2014 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0435, \u043A\u043E\u0442\u043E\u0440\u043E\u0435 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0437\u0438\u0440\u0443\u0435\u0442 \u0437\u0430\u0440\u0430\u0431\u043E\u0442\u043E\u043A \u043D\u0430 \u043F\u043E\u0441\u0442\u0430\u0432\u043A\u0435 \u043B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u0438. \u041F\u0440\u043E\u0435\u043A\u0442 \u043D\u0430\u0447\u0430\u043B \u0440\u0430\u0431\u043E\u0442\u0443 \u0432 \u0444\u0435\u0432\u0440\u0430\u043B\u0435 2020 \u0433\u043E\u0434\u0430 \u0438 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438 \u043F\u0435\u0440\u0435\u043C\u0435\u0449\u0430\u043B \u043C\u043E\u043D\u0435\u0442\u044B \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u0439 \u0432 \u043F\u0443\u043B \u043B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u0438 \u0441 \u043D\u0430\u0438\u0431\u043E\u043B\u044C\u0448\u0438\u043C \u043F\u0440\u043E\u0446\u0435\u043D\u0442\u043E\u043C \u0434\u043E\u0445\u043E\u0434\u043D\u043E\u0441\u0442\u0438. \u0421 \u043F\u043E\u044F\u0432\u043B\u0435\u043D\u0438\u0435\u043C \xAB\u0414\u043E\u0445\u043E\u0434\u043D\u043E\u0433\u043E \u0444\u0435\u0440\u043C\u0435\u0440\u0441\u0442\u0432\u0430\xBB, \u0441\u043E\u0437\u0434\u0430\u0442\u0435\u043B\u044C Yearn Finance \u0410\u043D\u0434\u0440\u0435 \u041A\u0440\u043E\u043D\u044C\u0435 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0430\u043B \u0441\u0438\u0441\u0442\u0435\u043C\u0443 \u0445\u0440\u0430\u043D\u0438\u043B\u0438\u0449, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0443\u0447\u0438\u0442\u044B\u0432\u0430\u044E\u0442 \u043F\u0440\u0438\u0431\u044B\u043B\u044C \u0438 \u043E\u0442 \u043F\u0440\u043E\u0446\u0435\u043D\u0442\u043E\u0432 \u0434\u043E\u0445\u043E\u0434\u043D\u043E\u0441\u0442\u0438, \u0438 \u043E\u0442 \u0440\u0430\u0441\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u0438\u044F \u0442\u043E\u043A\u0435\u043D\u043E\u0432. \u0421\u0438\u0441\u0442\u0435\u043C\u0430 \u0445\u0440\u0430\u043D\u0438\u043B\u0438\u0449 \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u0441 \u0438\u044E\u043B\u044F 2020 \u0433\u043E\u0434\u0430.  \u0425\u0440\u043E\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0432\u044B\u0445\u043E\u0434\u0430 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432, \u0432\u0430\u0436\u043D\u044B\u0445 \u0434\u043B\u044F \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u044F DeFi \u0425\u0440\u043E\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0432\u044B\u0445\u043E\u0434\u0430 \u0432\u0430\u0436\u043D\u044B\u0445 \u0434\u043B\u044F DeFi \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u041D\u0430 Ethereum \u043E\u0442\u043D\u043E\u0441\u0438\u0442\u0435\u043B\u044C\u043D\u043E \u043F\u0440\u043E\u0441\u0442\u043E \u0432\u0437\u044F\u0442\u044C \u0437\u0430 \u043E\u0441\u043D\u043E\u0432\u0443 \u043E\u0442\u043A\u0440\u044B\u0442\u044B\u0439 \u0438\u0441\u0445\u043E\u0434\u043D\u044B\u0439 \u043A\u043E\u0434 \u0434\u0440\u0443\u0433\u043E\u0433\u043E \u043F\u0440\u043E\u0435\u043A\u0442\u0430, \u0434\u043E\u0440\u0430\u0431\u043E\u0442\u0430\u0442\u044C \u0435\u0433\u043E \u0438 \u0432\u044B\u043F\u0443\u0441\u0442\u0438\u0442\u044C \u0441\u043E\u0431\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0439. \u0422\u0430\u043A, \u043D\u0430\u0448\u0438 \u043F\u0430\u0440\u0442\u043D\u0435\u0440\u044B \u0438\u0437 Monolithos \u0432\u0437\u044F\u043B\u0438 \u0437\u0430 \u043E\u0441\u043D\u043E\u0432\u0443 \u0441\u0432\u043E\u0435\u0433\u043E \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F Land \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u043D\u044B\u0439 \u043A\u043E\u0434 MakerDAO. \u0420\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u0438 \u0438\u0437\u043C\u0435\u043D\u0438\u043B\u0438 \u0441\u0442\u0435\u0439\u0431\u043B\u043A\u043E\u0438\u043D, \u043A\u043E\u0442\u043E\u0440\u044B\u0439 \u0432\u044B\u043F\u0443\u0441\u043A\u0430\u0435\u0442 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0435. \u0412\u043C\u0435\u0441\u0442\u043E \u0432\u044B\u043F\u0443\u0441\u043A\u0430 DAI, \u043F\u0440\u0438\u0432\u044F\u0437\u0430\u043D\u043D\u043E\u0433\u043E \u043A \u0434\u043E\u043B\u043B\u0430\u0440\u0443 \u0421\u0428\u0410, \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0435 Land \u043F\u0440\u043E\u0435\u043A\u0442\u0430 MonolithosDAO \u0432\u044B\u043F\u0443\u0441\u043A\u0430\u0435\u0442 MCR, \u043F\u0440\u0438\u0432\u044F\u0437\u0430\u043D\u043D\u044B\u0439 \u043A \u0440\u043E\u0441\u0441\u0438\u0439\u0441\u043A\u043E\u043C\u0443 \u0440\u0443\u0431\u043B\u044E. \u0414\u0440\u0443\u0433\u043E\u0439 \u043F\u0440\u0438\u043C\u0435\u0440 \u2014 \u0434\u0435\u0446\u0435\u043D\u0442\u0440\u0430\u043B\u0438\u0437\u043E\u0432\u0430\u043D\u043D\u0430\u044F \u0431\u0438\u0440\u0436\u0430 SushiSwap. \u0415\u0435 \u0441\u043E\u0437\u0434\u0430\u0442\u0435\u043B\u0438 \u0432\u0437\u044F\u043B\u0438 \u0437\u0430 \u043E\u0441\u043D\u043E\u0432\u0443 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u043D\u044B\u0439 \u043A\u043E\u0434 Uniswap \u0438 \u0434\u043E\u0440\u0430\u0431\u043E\u0442\u0430\u043B\u0438 \u0441\u0438\u0441\u0442\u0435\u043C\u0443 \u0440\u0430\u0441\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u0438\u044F \u0443\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0449\u0438\u0445 \u0442\u043E\u043A\u0435\u043D\u043E\u0432 \u043F\u043E\u0441\u0442\u0430\u0432\u0449\u0438\u043A\u0430\u043C\u0438 \u043B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u0438.  \u0421\u043A\u043E\u043B\u044C\u043A\u043E DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u043D\u0430 Ethereum \u0427\u0442\u043E\u0431\u044B \u043E\u0446\u0435\u043D\u0438\u0442\u044C, \u043A\u0430\u043A\u043E\u0439 \u043F\u0440\u043E\u0446\u0435\u043D\u0442 \u0440\u044B\u043D\u043A\u0430 DeFi \u0437\u0430\u043D\u0438\u043C\u0430\u044E\u0442 \u043F\u0440\u043E\u0435\u043A\u0442\u044B \u043D\u0430 Ethereum, \u043E\u0431\u0440\u0430\u0442\u0438\u043C\u0441\u044F \u043A \u043E\u0442\u0447\u0435\u0442\u0443 dappradar.com \u0437\u0430 2020 \u0433\u043E\u0434. \u0421\u0440\u0430\u0432\u043D\u0438\u043C 3 \u043F\u043E\u043A\u0430\u0437\u0430\u0442\u0435\u043B\u044F:  \u0421\u043A\u043E\u043B\u044C\u043A\u043E DeFi-\u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0439 \u0431\u044B\u043B\u043E \u0432\u044B\u043F\u0443\u0449\u0435\u043D\u043E \u0437\u0430 2020 \u0433\u043E\u0434 \u043D\u0430 Ethereum \u0438 \u043D\u0430 \u0434\u0440\u0443\u0433\u0438\u0445 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0430\u0445. \u041A\u0430\u043A\u043E\u0439 \u043E\u0431\u044A\u0435\u043C \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0432 DeFi \u043D\u0430 Ethereum \u0438 \u043D\u0430 \u0434\u0440\u0443\u0433\u0438\u0445 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0430\u0445. \u041A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0430\u043A\u0442\u0438\u0432\u043D\u044B\u0445 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u0439. \u041A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0432\u044B\u043F\u0443\u0449\u0435\u043D\u043D\u044B\u0445 DeFi-\u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0439. \u0412 2020 \u0433\u043E\u0434\u0443 \u0431\u044B\u043B\u043E \u0432\u044B\u043F\u0443\u0449\u0435\u043D\u043E 238 DeFi-\u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F. \u0418\u0437 \u043D\u0438\u0445 106 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0440\u0430\u0431\u043E\u0442\u0430\u044E\u0442 \u043D\u0430 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435 Ethereum \u2014 \u044D\u0442\u043E 45% \u043E\u0442 \u043E\u0431\u0449\u0435\u0433\u043E \u0447\u0438\u0441\u043B\u0430 \u043D\u043E\u0432\u044B\u0445 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432.  \u0414\u043E\u043B\u044F Ethereum \u0432 DeFi-\u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F\u0445 \u0438 \u043E\u0431\u044A\u0435\u043C\u0435 \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0432 DeFI \u041E\u0431\u044A\u0435\u043C \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0438 \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u043D\u043E\u0432\u044B\u0445 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0439 \u0432 DeFi \u043D\u0430 Ethereum \u0438 \u043D\u0430 \u0434\u0440\u0443\u0433\u0438\u0445 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0430\u0445 \u041E\u0431\u044A\u0435\u043C \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0432 DeFi-\u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F\u0445. \u0412 2020 \u0433\u043E\u0434\u0443 \u043E\u0431\u0449\u0438\u0439 \u043E\u0431\u044A\u0435\u043C \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0432 DeFi \u0441\u043E\u0441\u0442\u0430\u0432\u0438\u043B $271 \u043C\u043B\u0440\u0434. \u0418 95% \u043E\u0442 \u044D\u0442\u043E\u0433\u043E \u043E\u0431\u044A\u0435\u043C\u0430 \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0431\u044B\u043B\u043E \u043F\u0440\u043E\u0432\u0435\u0434\u0435\u043D\u043E \u043D\u0430 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435 Ethereum. \u0421\u0440\u0430\u0432\u043D\u0438\u043C \u043E\u0431\u044A\u0435\u043C \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0437\u0430 2020 \u0433\u043E\u0434 \u0432 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u0430\u0445 \u0440\u0430\u0437\u043D\u044B\u0445 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u043E\u0432:  Ethereum \u2014 $256 \u043C\u043B\u0440\u0434; EOS \u2014 $6,5 \u043C\u043B\u0440\u0434; TRON \u2014 $3,6 \u043C\u043B\u0440\u0434; Binance Smart Chain \u2014 $3 \u043C\u043B\u0440\u0434. \u041A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0430\u043A\u0442\u0438\u0432\u043D\u044B\u0445 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u0439. \u0421\u0440\u0435\u0434\u043D\u0435\u0435 \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0430\u043A\u0442\u0438\u0432\u043D\u044B\u0445 \u043A\u043E\u0448\u0435\u043B\u044C\u043A\u043E\u0432 \u0432\u043E \u0432\u0441\u0435\u0445 \u0434\u0435\u0446\u0435\u043D\u0442\u0440\u0430\u043B\u0438\u0437\u043E\u0432\u0430\u043D\u043D\u044B\u0445 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F\u0445 \u0441\u043E\u0441\u0442\u0430\u0432\u0438\u043B\u043E 135 \u0442\u044B\u0441. \u0418\u0437 \u043D\u0438\u0445 39 \u0442\u044B\u0441 \u0438\u043B\u0438 29%  \u2014 \u043D\u0430 Ethereum. \u0417\u0434\u0435\u0441\u044C \u043E\u0442\u043C\u0435\u0442\u0438\u043C, \u0447\u0442\u043E \u044D\u0442\u043E \u0446\u0438\u0444\u0440\u044B \u043F\u043E \u0430\u043A\u0442\u0438\u0432\u043D\u044B\u043C \u043A\u043E\u0448\u0435\u043B\u044C\u043A\u0430\u043C \u0432\u043E \u0432\u0441\u0435\u0445 \u043D\u0430\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u0438\u044F\u0445 \u0434\u0435\u0446\u0435\u043D\u0442\u0440\u0430\u043B\u0438\u0437\u043E\u0432\u0430\u043D\u043D\u044B\u0445 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0439, \u043D\u0435 \u0442\u043E\u043B\u044C\u043A\u043E \u0432 DeFi. \u041D\u043E 99% \u043E\u0442 \u0432\u0441\u0435\u0433\u043E \u043E\u0431\u044A\u0435\u043C\u0430 \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0432 \u044D\u0442\u0438\u0445 \u043A\u043E\u0448\u0435\u043B\u044C\u043A\u0430\u0445 \u0441\u0432\u044F\u0437\u0430\u043D\u044B \u0438\u043C\u0435\u043D\u043D\u043E \u0441 \u044D\u043A\u043E\u0441\u0438\u0441\u0442\u0435\u043C\u043E\u0439 DeFi.  \u0417\u0430\u043A\u043B\u044E\u0447\u0435\u043D\u0438\u0435 \u0421\u0443\u0449\u0435\u0441\u0442\u0432\u0443\u0435\u0442 \u043D\u0435\u0441\u043A\u043E\u043B\u044C\u043A\u043E \u043F\u0440\u0438\u0447\u0438\u043D, \u043F\u043E \u043A\u043E\u0442\u043E\u0440\u044B\u043C \u0431\u043E\u043B\u044C\u0448\u0438\u043D\u0441\u0442\u0432\u043E DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432 \u0441\u043E\u0437\u0434\u0430\u044E\u0442\u0441\u044F \u043D\u0430 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435 Ethereum. \u0423 \u044D\u0442\u043E\u0433\u043E \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0430 \u0441\u0444\u043E\u0440\u043C\u0438\u0440\u043E\u0432\u0430\u043B\u0430\u0441\u044C \u0438\u043D\u0444\u0440\u0430\u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u0430, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u0432\u043B\u0438\u044F\u0435\u0442 \u043D\u0430 \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u0435 DeFi. \u041E\u043D\u0430 \u0432\u043A\u043B\u044E\u0447\u0430\u0435\u0442 \u0432 \u0441\u0435\u0431\u044F \u043D\u0435\u043A\u043E\u043C\u043C\u0435\u0440\u0447\u0435\u0441\u043A\u0438\u0435 \u043E\u0440\u0433\u0430\u043D\u0438\u0437\u0430\u0446\u0438\u0438, \u0430\u0443\u0434\u0438\u0442\u043E\u0440\u0441\u043A\u0438\u0435 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438 \u0438 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u044B, \u0432 \u043A\u043E\u0442\u043E\u0440\u044B\u0445 \u043C\u043E\u0436\u043D\u043E \u0437\u0430\u0441\u0442\u0440\u0430\u0445\u043E\u0432\u0430\u0442\u044C \u0441\u0432\u043E\u0438 \u0441\u0440\u0435\u0434\u0441\u0442\u0432\u0430 \u0432 \u0434\u0440\u0443\u0433\u0438\u0445 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F\u0445.  \u041F\u0435\u0440\u0432\u044B\u0439 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442 \u0437\u0430\u043F\u0443\u0441\u0442\u0438\u043B\u0441\u044F \u043D\u0430 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435 Ethereum \u0432 \u043A\u043E\u043D\u0446\u0435 2017 \u0433\u043E\u0434\u0430. \u0412 \u0434\u0430\u043B\u044C\u043D\u0435\u0439\u0448\u0435\u043C, \u0432\u0430\u0436\u043D\u044B\u0435 \u0434\u043B\u044F \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u044F DeFi \u043F\u0440\u043E\u0435\u043A\u0442\u044B \u0432\u044B\u0445\u043E\u0434\u0438\u043B\u0438 \u0442\u0430\u043A\u0436\u0435 \u043D\u0430 \u044D\u0442\u043E\u043C \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435:  MakerDAO \u2014 \u043F\u0435\u0440\u0432\u044B\u0439 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442; Uniswap \u2014 \u043F\u0435\u0440\u0432\u044B\u0439 \u0443\u0441\u043F\u0435\u0448\u043D\u044B\u0439 \u043F\u0440\u0438\u043C\u0435\u0440 \u0440\u0435\u0430\u043B\u0438\u0437\u0430\u0446\u0438\u0438 DEX; Compound \u2014 \u0437\u0430\u0434\u0430\u043B \u0442\u0440\u0435\u043D\u0434 \u043D\u0430 \xAB\u0414\u043E\u0445\u043E\u0434\u043D\u043E\u0435 \u0444\u0435\u0440\u043C\u0435\u0440\u0441\u0442\u0432\u043E\xBB; Yearn Finance \u2014 \u0438\u043D\u0441\u0442\u0440\u0443\u043C\u0435\u043D\u0442 \u0434\u043B\u044F \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0437\u0430\u0446\u0438\u0438 \u0437\u0430\u0440\u0430\u0431\u043E\u0442\u043A\u0430 \u043D\u0430 \u043F\u043E\u0441\u0442\u0430\u0432\u043A\u0430\u0445 \u043B\u0438\u043A\u0432\u0438\u0434\u043D\u043E\u0441\u0442\u0438 \u0438 \u043D\u0430 \xAB\u0414\u043E\u0445\u043E\u0434\u043D\u043E\u043C \u0444\u0435\u0440\u043C\u0435\u0440\u0441\u0442\u0432\u0435\xBB. \u041E\u0434\u043D\u043E \u0438\u0437 \u0433\u043B\u0430\u0432\u043D\u044B\u0445 \u043A\u0430\u0447\u0435\u0441\u0442\u0432 Ethereum \u2014 \u043F\u0440\u043E\u0441\u0442\u043E\u0442\u0430 \u044F\u0437\u044B\u043A\u0430 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F Solidity. \u0423 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u043E\u0432 \u0435\u0441\u0442\u044C \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E\u0441\u0442\u044C \u0431\u0440\u0430\u0442\u044C \u0437\u0430 \u043E\u0441\u043D\u043E\u0432\u0443 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u043D\u044B\u0439 \u043A\u043E\u0434 \u0443\u0441\u043F\u0435\u0448\u043D\u044B\u0445 \u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432, \u0432\u043D\u043E\u0441\u0438\u0442\u044C \u0432 \u043D\u0438\u0445 \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u044F, \u0443\u043B\u0443\u0447\u0448\u0435\u043D\u0438\u044F \u0438 \u0432\u044B\u043F\u0443\u0441\u043A\u0430\u0442\u044C \u043D\u043E\u0432\u044B\u0439 \u043F\u0440\u043E\u0434\u0443\u043A\u0442.   \u041F\u043E \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0430\u043C \u043E\u0442\u0447\u0435\u0442\u0430 dappradar \u0437\u0430 2020 \u0433\u043E\u0434, 95% \u0432\u0441\u0435\u0433\u043E \u043E\u0431\u044A\u0435\u043C\u0430 \u0442\u0440\u0430\u043D\u0437\u0430\u043A\u0446\u0438\u0439 \u0432 DeFI \u043F\u0440\u043E\u0438\u0441\u0445\u043E\u0434\u0438\u043B\u0438 \u043D\u0430 \u0431\u043B\u043E\u043A\u0447\u0435\u0439\u043D\u0435 Ethereum. \u0417\u0430 \u0433\u043E\u0434 \u0431\u044B\u043B\u043E \u0432\u044B\u043F\u0443\u0449\u0435\u043D\u043E 238 DeFi-\u043F\u0440\u043E\u0435\u043A\u0442\u043E\u0432, 45% \u0438\u0437 \u043A\u043E\u0442\u043E\u0440\u044B\u0445 \u0440\u0430\u0431\u043E\u0442\u0430\u044E\u0442 \u043D\u0430 Ethereum.  \u0412 \u0441\u043B\u0435\u0434\u0443\u044E\u0449\u0435\u043C \u043C\u0430\u0442\u0435\u0440\u0438\u0430\u043B\u0435 \u043C\u044B \u0440\u0430\u0441\u0441\u043A\u0430\u0436\u0435\u043C, \u043A\u0430\u043A\u0438\u0435 \u0432\u0438\u0434\u044B DeFi-\u0442\u043E\u043A\u0435\u043D\u043E\u0432 \u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0443\u044E\u0442 \u0438 \u043F\u0440\u0438\u0432\u0435\u0434\u0435\u043C \u043F\u0440\u0438\u043C\u0435\u0440\u044B." #@param {type:"string"}
stexts = texts
print(texts)

Почему большинство DeFi-проектов создаются на Ethereum Опубликовано: 17/02/2021Автор: Дмитрий ЕгоровКатегория: Карта сокровищ DeFi, НовичокОставить комментарий Почему большинство DeFi-проектов создаются на Ethereum Обложка Содержание	 Вспоминаем основные понятия Кто обслуживает DeFi на Ethereum Ethereum и развитие DeFi Сколько DeFi-проектов работает на Ethereum Заключение Совместно с партнерами из Monolithos, мы продолжаем цикл материалов про децентрализованные финансы. Сегодня мы разберемся, почему большинство DeFi-проектов работают на блокчейне Ethereum.  Деньги под управлением программ: Что такое DeFi Зачем использовать DeFi, если есть банки Что общего у DeFi и криптовалюты Почему большинство DeFi-проектов создаются на Ethereum Вспоминаем основные понятия Смарт-контракт — программа, которая автоматически совершает действия при достижении определенных условий. Управляющие токены — токены, которые дают право голоса за внесение изменений в приложение. Доходное фермерство — стратегия за

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [ ]:
def printTF_IDF(texts):
  records_count = 30
  tfIdfTransformer = TfidfVectorizer(ngram_range=(1, 4), use_idf=True, max_features=records_count)
  countVectorizer = CountVectorizer(ngram_range=(1, 4), max_features=records_count)
  wordCount = countVectorizer.fit_transform([texts])
  TfIdf = tfIdfTransformer.fit_transform([texts])
  names = countVectorizer.get_feature_names()
  df=[]
  df = pd.DataFrame(list(names), columns=['names'])
  df = df.assign(Word_Count=wordCount.T.todense())
  df = df.assign(TF_IDF=TfIdf.T.todense())
  df = df.sort_values('TF_IDF', ascending=False)
  print(df)

In [ ]:
printTF_IDF(texts)

                    names  Word_Count    TF_IDF
13                     на          65  0.548841
1                    defi          60  0.506622
3                ethereum          47  0.396854
23               проектов          32  0.270199
14            на ethereum          24  0.202649
7                     для          23  0.194205
2           defi проектов          15  0.126656
9                      за          11  0.092881
4               блокчейне          10  0.084437
6                    года          10  0.084437
26                  смарт          10  0.084437
20                     по          10  0.084437
0                    2020           9  0.075993
28                токенов           9  0.075993
22                проекта           9  0.075993
29             транзакций           9  0.075993
10                     из           9  0.075993
12                которые           8  0.067550
21                 проект           8  0.067550
24            проектов на           8  0

удалим знаки переноса строк и сделаем текст в нижнем регистре

In [ ]:
  str2 = ''
  for item in texts.split():
    str2 = str2 + ' ' + item
  texts = str(texts)
  texts = texts.lower()
  # print(texts)
  texts = texts.replace('\n', ' ')

удалим цифры

In [ ]:
  str2 = ''
  for c in texts:
      if c not in ('0', "1", '2', '3', '4', '5', '6', '7', '8', '9', '«', '»', '–', "\""):
          str2 = str2 + c
  texts = str2
  str2 = ''

удалим знаки пунктуации

In [ ]:
import string
pattern = string.punctuation
for c in texts:
    if c not in pattern:
        str2 = str2 + c
    else:
        str2 = str2 + " "
texts = str2
str2 = ''

удалим стоп слова

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
  from nltk.corpus import stopwords
  russian_stopwords = stopwords.words("russian")
  for word in texts.split():
      if word not in (russian_stopwords):
          str2 = str2 + " " + word
  texts = str2

  str2 = ''
  for word in texts.split():
      if len(word) > 1:
          str2 = str2 + " " + word
  texts = str2
  print(texts)

 почему большинство defi проектов создаются ethereum опубликовано автор дмитрий егоровкатегория карта сокровищ defi новичокоставить комментарий почему большинство defi проектов создаются ethereum обложка содержание вспоминаем основные понятия обслуживает defi ethereum ethereum развитие defi сколько defi проектов работает ethereum заключение совместно партнерами monolithos продолжаем цикл материалов децентрализованные финансы сегодня разберемся почему большинство defi проектов работают блокчейне ethereum деньги управлением программ такое defi использовать defi банки общего defi криптовалюты почему большинство defi проектов создаются ethereum вспоминаем основные понятия смарт контракт программа которая автоматически совершает действия достижении определенных условий управляющие токены токены которые дают право голоса внесение изменений приложение доходное фермерство стратегия заработка которая учитывает проценты доходности раздачу управляющих токенов ликвидность средства пользователей сч

рассчитаем TF-IDF

In [ ]:
printTF_IDF(texts)

                 names  Word_Count    TF_IDF
1                 defi          60  0.649380
3             ethereum          47  0.508681
23            проектов          32  0.346336
2        defi проектов          15  0.162345
26               смарт          10  0.108230
6            блокчейне          10  0.108230
9                 года          10  0.108230
29          транзакций           9  0.097407
28             токенов           9  0.097407
22             проекта           9  0.097407
24             проекты           8  0.086584
21              проект           8  0.086584
15             которые           8  0.086584
10              других           8  0.086584
13            компании           7  0.075761
27          страхового           7  0.075761
25       разработчиков           7  0.075761
7   блокчейне ethereum           7  0.075761
0                  dai           7  0.075761
18               объем           7  0.075761
17         обеспечения           7  0.075761
16        

#Суммаризации текстов 

Задача суммаризации текстов (автореферирование) - одна из ключевых, широко обсуждаемых задач NLP. Она состоит в сжатии больших объемов текста до связного краткого содержания, отражающего только основные идеи.


#Алгоритм TextRank
TextRank - это алгоритм, основанный на PageRank, который часто используется для извлечения ключевых слов и суммирования текста. 
PageRank (PR) - это алгоритм, используемый для расчета веса веб-страниц. 


In [ ]:
from gensim.summarization import summarize, keywords
print("Исходный текст:")
print(stexts)
print("Результат работы TextRank:")
print(summarize(str(stexts)))

Исходный текст:
Почему большинство DeFi-проектов создаются на Ethereum Опубликовано: 17/02/2021Автор: Дмитрий ЕгоровКатегория: Карта сокровищ DeFi, НовичокОставить комментарий Почему большинство DeFi-проектов создаются на Ethereum Обложка Содержание	 Вспоминаем основные понятия Кто обслуживает DeFi на Ethereum Ethereum и развитие DeFi Сколько DeFi-проектов работает на Ethereum Заключение Совместно с партнерами из Monolithos, мы продолжаем цикл материалов про децентрализованные финансы. Сегодня мы разберемся, почему большинство DeFi-проектов работают на блокчейне Ethereum.  Деньги под управлением программ: Что такое DeFi Зачем использовать DeFi, если есть банки Что общего у DeFi и криптовалюты Почему большинство DeFi-проектов создаются на Ethereum Вспоминаем основные понятия Смарт-контракт — программа, которая автоматически совершает действия при достижении определенных условий. Управляющие токены — токены, которые дают право голоса за внесение изменений в приложение. Доходное фермерств

#Алгоритм Rake
RAKE: Rapid Automatic Keyword Extraction Algorithm
Алгоритм RAKE извлекает ключевые слова с помощью основанного на разделителе подхода, чтобы идентифицировать ключевые слова кандидата и баллы их использующий совместные встречаемости слова, которые появляются в ключевых словах кандидата. Ключевые слова могут содержать несколько лексем. Кроме того, алгоритм RAKE также объединяет ключевые слова, когда они кажутся многократно, разделенными тем же разделителем слияния.


In [ ]:
!pip install rake_nltk
from rake_nltk import Metric, Rake
r = Rake(language="russian")
r.extract_keywords_from_text(stexts)
mas = r.get_ranked_phrases()
set2 = set()
for item in mas:
    if not "nan" in str(item).replace(" nan ", " "):
        set2.add(str(item).replace(" nan ", " "))
mas = list(set2)
print("Исходный текст:")
print(stexts)
print("Результат работы Rake:")
print(str(mas))

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=27a026b2ad95238eb2efc519ea767fbc93580c67bc72e0d3999652f43c8807c7
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk
Исходный текст:
Почему большинство DeFi-проектов создаются на Ethereum Опубликовано: 17/02/2021Автор: Дмитрий ЕгоровКатегория: Карта сокровищ DeFi, НовичокОставить комментарий Почему большинство DeFi-проектов создаются на Ethereum Обложка Содержание	 Вспоминаем основные понятия Кто обслуживает DeFi на Ethereum Ethereum и развитие DeFi Сколько DeFi-проектов работает на Ethereum Заключение Совместно с партнерами из Monolithos, мы продолжаем цикл материалов про децентрализованные финансы. Сегодня мы разберемся, почему большинство DeFi-проектов работают на блокчейне Ethereum.  Деньги под управлением программ: Что такое DeFi Зачем использовать DeFi, если есть банки Что общего у DeFi и криптовалюты Поч

#Тематическое моделирование
Тематическое моделирование — это метод извлечения тем из текста. Latent Dirichlet Allocation (LDA) — популярный алгоритм моделирования тем реализованные в том числе в пакете Gensim. Основная задача алгоритмов ТМ, заключается в том что бы полученные темы были хорошего качество, понятными, самозначимыми и разделенными. Достижение этих целей во многом зависит от качества предварительной обработки текста и стратегии поиска оптимального количества тем. 

##Импорт пакетов


In [ ]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Подход LDA к тематическому моделированию заключается в том, что каждый документ рассматривается как набор тем в определенной пропорции. И каждая тема как набор ключевых слов, опять же, в определенной пропорции.

После того, как вы укажете алгоритму количество тем, все, что он сделает, — это отобразит распределение тем в документах и распределение ключевых слов по темам.

Тема — это не что иное, как набор доминирующих ключевых слов. Просто взглянув на ключевые слова, вы сможете определить, о чем эта тема.

Ниже приведены ключевые факторы для получения хороших разделительных тем:

Качество обработки текста.
Разнообразие тем, о которых говорится в тексте.
Выбор алгоритма моделирование тем.
Количество тем, указанных в алгоритме.
Алгоритмы настройки параметров.

# Подготовим стоп-слова


In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

##Лемматизация
Лемматизация — это не что иное, как преобразование слова в его корневое слово. Например: лемма слова «machines» — это «machine». Аналогично, «walking» -> «walk», «mice» -> «mouse» и так далее.

In [ ]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 1:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 8.2MB 7.3MB/s 
 почему большинство defi проектов создаются ethereum опубликовано автор дмитрий егоровкатегория карта сокровищ defi новичокоставить комментарий почему большинство defi проектов создаются ethereum обложка содержание вспоминаем основные понятия обслуживает defi ethereum ethereum развитие defi сколько defi проектов работает ethereum заключение совместно партнерами monolithos продолжаем цикл материалов децентрализованные финансы сегодня разберемся почему большинство defi проектов работают блокчейне ethereum деньги управлением программ такое defi использовать defi банки общего defi криптовалюты почему большинство defi проектов создаются ethereum вспоминаем основные понятия смарт контракт программа которая автоматически совершает действия достижении определенных условий управляющие токены токены которые дают право голоса внесение изменений приложение доходное фермерство стратегия зара

##Создадим словарь и корпус.
Двумя основными входными данными для тематической модели LDA являются словарь (id2word) и корпус. Давайте создадим их.

In [ ]:
from nltk.util import ngrams
def make_bigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    bigrams = list(ngrams(token, 2))
    # print(bigrams)
    return bigrams
def make_trigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    trigrams = list(ngrams(token, 2))
    return trigrams

btexts = make_bigrams(texts) + make_trigrams(texts)
id2word = corpora.Dictionary(btexts)
corpus = [id2word.doc2bow(text) for text in btexts]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('большинство', 1), ('почему', 1)]]

Или вы можете увидеть удобочитаемую форму самого корпуса.

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('большинство', 1), ('почему', 1)]]

##Построим тематическую модель
У нас есть все необходимое для обучения модели LDA. В дополнение к корпусу и словарю необходимо также указать количество тем.

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

##Просмотр тем в модели LDA
Вышеупомянутая модель LDA построена на 20 различных темах, где каждая тема представляет собой комбинацию ключевых слов, и каждое ключевое слово вносит определенный вес в тему.

Вы можете увидеть ключевые слова для каждой темы и вес (важность) каждого ключевого слова, используя lda_model.print_topics().

In [ ]:
lda_model.print_topics()

[(0,
  '0.045*"токенов" + 0.043*"год" + 0.042*"других" + 0.038*"ликвидности" + 0.036*"проекты" + 0.030*"пользователей" + 0.028*"проекта" + 0.026*"разработчиков" + 0.025*"блокчейнах" + 0.023*"тыс"'),
 (1,
  '0.026*"код" + 0.025*"проектов" + 0.023*"solidity" + 0.023*"uniswap" + 0.021*"приложение" + 0.021*"застраховать" + 0.020*"пример" + 0.019*"которые" + 0.018*"компании" + 0.018*"makerdao"'),
 (2,
  '0.186*"ethereum" + 0.043*"проект" + 0.039*"года" + 0.037*"млрд" + 0.024*"объема" + 0.021*"развития" + 0.020*"приложениях" + 0.016*"общий" + 0.016*"транзакций" + 0.014*"сравним"'),
 (3,
  '0.211*"defi" + 0.066*"проектов" + 0.047*"блокчейне" + 0.028*"первый" + 0.024*"приложений" + 0.022*"основу" + 0.018*"транзакций" + 0.018*"dai" + 0.014*"приложении" + 0.014*"работают"'),
 (4,
  '0.045*"объем" + 0.043*"количество" + 0.032*"транзакций" + 0.031*"смарт" + 0.028*"активных" + 0.022*"приложения" + 0.021*"finance" + 0.021*"страхового" + 0.021*"compound" + 0.019*"контрактов"')]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=stexts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

##Визуализация темы и ключевых слов
Теперь, когда модель LDA создана, следующим шагом является изучение созданных тем и связанных с ними ключевых слов. Нет лучшего инструмента, чем интерактивная диаграмма пакета pyLDAvis.

In [ ]:
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

     |████████████████████████████████| 1.7MB 4.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=08721a763dfe3f8acc08633853ba3db6f4d86f6dc3e781805bb0caf3f45af53e
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.190844 -0.209647       1        1  23.146110
1     -0.186707  0.085150       2        1  21.395779
0      0.029316  0.222845       3        1  19.292945
2      0.227413 -0.176021       4        1  18.600224
4      0.120822  0.077674       5        1  17.564942, topic_info=            Term        Freq       Total Category  logprob  loglift
2           defi  225.000000  225.000000  Default  30.0000  30.0000
5       ethereum  160.000000  160.000000  Default  29.0000  29.0000
3       проектов   95.000000   95.000000  Default  28.0000  28.0000
35     блокчейне   50.000000   50.000000  Default  27.0000  27.0000
72       токенов   40.000000   40.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
285  обеспечения   12.689461   18.085578   Topic5  -4.1529   1.3849
444        yearn   12.823321   18.393275   Topic5  -4.1424   1.3785
509   транзакций   26.040745   59.404672   Topic5  -3.4340   0.9146
192       аудиты    7.485168   11.669142   Topic5  -4.6808   1.2952
404        тренд    6.125455   12.545071   Topic5  -4.8812   1.0224

[202 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
525       4  0.838673    binance
527       4  0.838673      chain
360       2  0.971440        com
402       5  0.984252   compound
198       4  0.946926  consensys
...     ...       ...        ...
512       1  0.805546      числа
234       5  0.962050       этих
159       2  0.933487        это
366       2  0.472574      языка
366       3  0.472574      языка

[183 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3, 5])